# Mühle GUI

In [ ]:
import ipycanvas
import math

In [ ]:
import logging

# create logger with ''
logger = logging.getLogger('GUI')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)

In [ ]:
# TODO: create schema to visualize following constants

# Board constants
BOARD_SIZE = 500
CANVAS_PADDING = 30
ROW_WIDTH = 60
PLAYER_PIECE_RADIUS = 12
DEFAULT_PIECE_RADIUS = 5

DEFAULT_STONE_LINE_WIDTH = 1
SELECTED_STONE_LINE_WIDTH = 3

STASH_STONES_SPACING = 5

STASH_HEIGHT = ((PLAYER_PIECE_RADIUS * 2 ) * 2) + (STASH_STONES_SPACING) + (CANVAS_PADDING * 2)
STASH_WIDTH = ((PLAYER_PIECE_RADIUS * 2 ) * 9) + (STASH_STONES_SPACING * 8) + (CANVAS_PADDING * 2)

CANVAS_HEIGHT = BOARD_SIZE
CANVAS_WIDTH = BOARD_SIZE + STASH_WIDTH

STASH_STARTING_POINT_X = BOARD_SIZE
STASH_STARTING_POINT_Y = CANVAS_HEIGHT - STASH_HEIGHT

# text
TEXT_X = BOARD_SIZE + CANVAS_PADDING
TEXT_Y = CANVAS_PADDING
TEXT_MAX_WIDTH = CANVAS_WIDTH - BOARD_SIZE - 2 * CANVAS_PADDING


# players
NO_PLAYER = ' '
PLAYER_1 = 'w'
PLAYER_2 = 'b'


# color constants
BOARD_FOREGROUND_COLOR = '#191919'
BOARD_BACKGROUND_COLOR = '#ffffcb'
STASH_BACKGROUND_COLOR = '#dedede'
PLAYER_1_FILL_COLOR = '#E8E8E8'
PLAYER_1_STROKE_COLOR = '#191919'
PLAYER_2_FILL_COLOR = '#3D3D3D'
PLAYER_2_STROKE_COLOR = '#191919'
TEXT_COLOR = '#ffffff'

# shadow constants
SHADOW_COLOR = '#000000'
SHADOW_OFFSET_X = 2
SHADOW_OFFSET_Y = 2
SHADOW_BLUR = 2

# default shadow constants
SHADOW_COLOR_DEFAULT = 'rgba(0, 0, 0, 0)'
SHADOW_OFFSET_X_DEFAULT = 0
SHADOW_OFFSET_Y_DEFAULT = 0
SHADOW_BLUR_DEFAULT = 0


In [ ]:
# all available values for x and y, beginning left to right and top to bottom
av = [
    math.floor(CANVAS_PADDING),
    math.floor(CANVAS_PADDING + ROW_WIDTH),
    math.floor(CANVAS_PADDING + 2 * ROW_WIDTH),
    math.floor(BOARD_SIZE / 2),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + 2 * ROW_WIDTH)),
    math.floor(BOARD_SIZE - (CANVAS_PADDING + ROW_WIDTH)),
    math.floor(BOARD_SIZE - CANVAS_PADDING)
]


In [ ]:
# All coordinates. The first tuple is the outer square, the second tuple is the middle square and the third tuple is the inner square. It always starts from top left and then clockwise.
coords = (
    (
        (av[0], av[0]),
        (av[3], av[0]),
        (av[6], av[0]),
        (av[6], av[3]),
        (av[6], av[6]),
        (av[3], av[6]),
        (av[0], av[6]),
        (av[0], av[3])
    ),
    (
        (av[1], av[1]),
        (av[3], av[1]),
        (av[5], av[1]),
        (av[5], av[3]),
        (av[5], av[5]),
        (av[3], av[5]),
        (av[1], av[5]),
        (av[1], av[3])
    ),
    (
        (av[2], av[2]),
        (av[3], av[2]),
        (av[4], av[2]),
        (av[4], av[3]),
        (av[4], av[4]),
        (av[3], av[4]),
        (av[2], av[4]),
        (av[2], av[3])
    )
) 

In [ ]:
# Function to draw a line on a canvas c, from start to end. Start and end are tuples with x and y values.
def drawLine(c, start, end):
    c.move_to(start[0], start[1])
    c.line_to(end[0], end[1])

In [ ]:
# Function to enable or disable shadows on a given canvas
def toggleShadow(c, enable):
    c.shadow_color    = SHADOW_COLOR    if enable == True else SHADOW_COLOR_DEFAULT
    c.shadow_offset_x = SHADOW_OFFSET_X if enable == True else SHADOW_OFFSET_X_DEFAULT
    c.shadow_offset_y = SHADOW_OFFSET_Y if enable == True else SHADOW_OFFSET_Y_DEFAULT
    c.shadow_blur     = SHADOW_BLUR     if enable == True else SHADOW_BLUR_DEFAULT

In [ ]:
# Function to draw a circle on a canvas c with given coords, radius and color
def drawCircle(c, coords, radius, color, strokeColor = None, useShadow = False, selected = False):
    if useShadow == True:
        toggleShadow(c, True)
    c.fill_style = color
    c.fill_arc(coords[0], coords[1], radius, 0, 2 * math.pi)
    if useShadow == True:
        toggleShadow(c, False)
    if strokeColor is not None:
        c.line_width = SELECTED_STONE_LINE_WIDTH if selected == True else DEFAULT_STONE_LINE_WIDTH
        c.strokeColor = strokeColor
        c.stroke_arc(coords[0], coords[1], radius, 0, 2 * math.pi)

In [ ]:
# Function to draw a (player) stone with given coords and for a given player (or the the default )
def drawStone(c, coords, player, selected = False):
    if player == PLAYER_1:
        drawCircle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_1_FILL_COLOR, strokeColor = PLAYER_1_STROKE_COLOR, useShadow = True, selected = selected)
        drawCircle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_1_FILL_COLOR, strokeColor = PLAYER_1_STROKE_COLOR, selected = selected)
    elif player == PLAYER_2:
        drawCircle(c, coords, PLAYER_PIECE_RADIUS, PLAYER_2_FILL_COLOR, strokeColor = PLAYER_2_STROKE_COLOR, useShadow = True, selected = selected)
        drawCircle(c, coords, math.floor(PLAYER_PIECE_RADIUS / 2), PLAYER_2_FILL_COLOR, strokeColor = PLAYER_2_STROKE_COLOR, selected = selected)
    else:
        drawCircle(c, coords, DEFAULT_PIECE_RADIUS, BOARD_FOREGROUND_COLOR)

In [ ]:
def drawText(c, text):
    c.clear()
    c.fill_text(text, TEXT_X, TEXT_Y, max_width = TEXT_MAX_WIDTH)

In [ ]:
# Function to construct a square on a canvas c.
def constructSquare(c, square):
    for i in range(4):
        start = i * 2
        end = (i * 2 + 2) if (i * 2 + 2 <= 6) else 0 
        drawLine(c, square[start], square[end])

In [ ]:
# Function to construct the cross lines on a canvas c
def constructCrossLines(c, coords):
    for i in range(4):
        k = i * 2 + 1
        drawLine(c, coords[0][k], coords[2][k])

In [ ]:
# Function to setup a canvas, returns a reference to a MultiCanvas
def setupCanvas():
    # level 0: Background and lines
    # level 1: Stones
    # level 2: Text
    canvas = ipycanvas.MultiCanvas(3, width = CANVAS_WIDTH, height = CANVAS_HEIGHT)
    with ipycanvas.hold_canvas(canvas[0]):
        canvas[0].fill_style = BOARD_BACKGROUND_COLOR
        canvas[0].fill_rect(0, 0, BOARD_SIZE, BOARD_SIZE)

        canvas[0].fill_style = STASH_BACKGROUND_COLOR
        canvas[0].fill_rect(STASH_STARTING_POINT_X, STASH_STARTING_POINT_Y, CANVAS_WIDTH, CANVAS_HEIGHT)


        canvas[0].stroke_style = BOARD_FOREGROUND_COLOR
        canvas[0].begin_path()

        for i in range(3):
            constructSquare(canvas[0], coords[i])

        constructCrossLines(canvas[0], coords)
        canvas[0].stroke()
    
    canvas[1].fill_style = TEXT_COLOR
    
    return canvas

In [ ]:
def updateGui(c, state, selectedStone = None):
    with ipycanvas.hold_canvas(c):
        c.clear()
        ((stashP1, stashP2), squares) = state

        # update pieces on the board
        for i in range(len(squares)):
            for j in range(len(squares[i])):
                drawStone(c, coords[i][j], squares[i][j], selected = ((selectedStone is not None) and (selectedStone[0] == i) and (selectedStone[1] ==j)))

        # update pieces on the stash

        # player 1

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + PLAYER_PIECE_RADIUS

        for i in range(stashP1):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_1)

        # player 2

        x = STASH_STARTING_POINT_X + PLAYER_PIECE_RADIUS
        y = STASH_STARTING_POINT_Y + CANVAS_PADDING + 3 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING

        for i in range(stashP2):
            x += 2 * PLAYER_PIECE_RADIUS + STASH_STONES_SPACING
            drawStone(c, (x, y), PLAYER_2)
    

## Game

In [ ]:
%run ./muehle-game.ipynb

In [ ]:
%run ./muehle-game-utils.ipynb

In [ ]:
# Test States
s1 = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))
s2 = ((0, 0), (
    ('w', 'w', ' ', 'b', 'w', 'b', 'w', 'w'),
    ('b', 'w', ' ', 'b', 'w', ' ', ' ', 'w'),
    ('b', 'b', 'b', 'b', ' ', ' ', 'b', 'w'),
))
s3 = ((7, 7), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))
s4 = ((2, 8), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))
s5 = ((0, 0), (('w', 'w', ' ', 'w', 'w', 'b', ' ', 'b'), (' ', 'b', 'w', ' ', 'w', 'w', 'b', 'b'), ('b', 'b', 'b', 'w', ' ', ' ', 'b', ' ')))

In [ ]:
def getClickedStone(x, y):
    for value in av:
        if value - PLAYER_PIECE_RADIUS <= x <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for x = {x} found: {value}')
            x = value
        if value - PLAYER_PIECE_RADIUS <= y <= value + PLAYER_PIECE_RADIUS:
            logger.info(f'value for y = {y} found: {value}')
            y = value
    stone = None
    for i in range(len(coords)):
        if stone is not None:
            break
        for j in range(len(coords[i])):
            if coords[i][j] == (x,y):
                stone = (i, j)

    logger.info(f'stone clicked: {stone}')
    return stone

In [ ]:
def convertStateTupleToList(s):
    return [list(s[0]), [list(s[1][0]), list(s[1][1]), list(s[1][2])]]

def convertStateListToTuple(s):
    return (tuple(s[0]), (tuple(s[1][0]), tuple(s[1][1]), tuple(s[1][2])))

## Klasse GameState

(TODO: Erklärung)

In [ ]:
class GameState:
    def __init__(self, state = None):
        if state is None:
            state = ((9, 9), (
            (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
            (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
            (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
        ))
        self.state = state
        self.player = PLAYER_1
        self.canvas = setupCanvas()
        self.hasMuehleToExecute = False
        self.selectedStone = None
        self.stateTemp = None
        self.winner = None
        
        self.canvas[2].on_mouse_up(self.handleGame)
        
        updateGui(self.canvas[2], self.state)
        self.updateText()
        logger.info('game state initalized')
        
    def handleGame(self, x, y):
        
        if self.winner is not None:
            logger.warning('Game has ended!')
            return
        
        logger.info(f'({x}, {y})')
        
        phase = playerPhase(self.state, self.player)
        logger.info(f'player phase: {phase}')
        
        stone = getClickedStone(x, y)
        
        if stone is None:
            logger.warning('No stone was clicked!')
            return
        
        if self.hasMuehleToExecute == True:
            self.executeMuehle(stone)
        else:
            if self.selectedStone is not None:
                self.moveStoneTo(stone, jump = True if phase == 3 else False)
            else:
                if phase == 1:
                    self.playerPhaseOne(stone)
                else:
                    if phase == 2 or phase == 3:
                        self.selectStone(stone)

        if self.checkIfFinished() == True:
            self.updateText()
                    

    def playerPhaseOne(self, stone):
        if self.state[1][stone[0]][stone[1]] != NO_PLAYER:
            logger.warning(f'{stone} is not free')
            return

        # remove stone from stash and place to board
        newState = convertStateTupleToList(self.state)
        newState[0][0 if self.player == PLAYER_1 else 1] -= 1#
        newState[1][stone[0]][stone[1]] = self.player
        allAvailableStates = nextStates(self.state, self.player)
        newState = convertStateListToTuple(newState)

        if self.validateNewState(newState) == True:
            logging.info('stone placed')
        else:
            logger.info('NewState not in allAvailableStates, checking for new Muehlen ...')
            newMuehlen = self.checkNewMuehlen(newState)

        self.updateText()
        
        
    # Player phase 2 and 3
    def selectStone(self, stone):
        if self.state[1][stone[0]][stone[1]] != self.player:
            logger.warning(f'{stone} is not the own stone')
            return
        self.selectedStone = stone
        updateGui(self.canvas[2], self.state, selectedStone = self.selectedStone)
        
        self.updateText()
            
    def checkNewMuehlen(self, newState):
        oldMuehlen = findMuehlen(self.state[1], self.player)
        newMuehlen = countNewMuehlen(newState[1], oldMuehlen, self.player)

        if newMuehlen > 0:
            self.stateTemp = newState
            self.hasMuehleToExecute = True
            updateGui(self.canvas[2], self.stateTemp)
        return newMuehlen
    
    def executeMuehle(self, stone):
        if self.hasMuehleToExecute == False:
            logger.warning('Player has no Muehlen to execute!')
            return
        if self.state[1][stone[0]][stone[1]] != opponent(self.player):
            logger.warning(f'{stone} is not the opponent!')
            return
        
        newState = convertStateTupleToList(self.stateTemp)
        newState[1][stone[0]][stone[1]] = NO_PLAYER
        newState = convertStateListToTuple(newState)
        
        if self.validateNewState(newState):
            logger.info('success')
        else:
            logger.warning('Muehle could not be executed!')    
        
        self.updateText()
    
    def togglePlayer(self):
        self.player = PLAYER_1 if self.player == PLAYER_2 else PLAYER_2
        
    def validateNewState(self, newState):
        allAvailableStates = nextStates(self.state, self.player)
        if newState in allAvailableStates:
            logger.info('SUCCESS')
            
            self.state = newState
            self.stateTemp = None
            self.hasMuehleToExecute = False
            self.selectedStone = None
            self.togglePlayer()
            updateGui(self.canvas[2], self.state)
            return True
        else:
            logger.warning('state is not in allAvailableStates!')
            return False
        
        
    def moveStoneTo(self, coord, jump = False):
        logger.info(f'moveStoneTo({coord}, jump = {jump})')
        if self.state[1][coord[0]][coord[1]] != NO_PLAYER:
            logger.warning(f'{coord} is not free')
            return
        
        if jump == True or coord in findNeighboringEmptyCells(self.state[1], self.selectedStone):  
            newState = convertStateTupleToList(self.state)
            newState[1][self.selectedStone[0]][self.selectedStone[1]] = NO_PLAYER
            newState[1][coord[0]][coord[1]] = self.player
            newState = convertStateListToTuple(newState)
            logger.info(newState)
            
            if self.validateNewState(newState) == True:
                movement = 'jumped' if jump == True else 'moved'
                logger.info(f'Stone successfully {movement}!')
            else:
                logger.info('Round not finished, checking for new muehlen...')
                self.checkNewMuehlen(newState)
        else:
            logger.warning(f'{coord} is not a (free) neighbor of {self.selectedStone}!')
        self.updateText()
            
    def updateText(self):
        phase = playerPhase(self.state, self.player)
        if self.winner is None:
            message = f'Player {self.player}: '
            if self.hasMuehleToExecute == True:
                message += 'execute your Muehle.'
            else:
                if self.selectedStone is not None:
                    movement = 'move' if phase == 2 else 'jump'
                    message += f'{movement} your selected stone'
                else:
                    if phase == 1:
                         message += 'Place your stone.'
                    else:
                        if phase == 2 or phase == 3:
                            movement = 'move' if phase == 2 else 'jump'
                            message += f'select your stone you want to {movement}'      
        else:
            message = 'The game has endend: '
            if self.winner == NO_PLAYER:
                message += 'Tie'
            else:
                message += f'{self.winner} has won!'
        logging.info(message)
        drawText(self.canvas[1], message)
        
    def checkIfFinished(self):
        if finished(self.state, self.player) == True:
            status = utility(self.state, self.player)
            if status == 0:
                self.winner = NO_PLAYER
            else:
                self.winner = self.player if status == 1 else opponent(self.player)
            return True
        else:
            return False

In [ ]:
gameState = GameState()

In [ ]:
gameState.canvas